In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

input_shape = 15*8*300  # Assuming the data is reshaped to this format
file_number = 366

# Data Loader Function
def load_data(data_dir):
    X = []
    y = []
    
    for file in os.listdir(data_dir):
        if file.endswith('.csv'):
            file_path = os.path.join(data_dir, file)
            data = pd.read_csv(file_path, header=None).values.flatten()
            data = np.array(data)
            
            label = 1 if 'seizure' in file.lower() else 0
            y.append(label)
            X.append(data)
    
    X = np.array(X).reshape(file_number, input_shape,1)
    y = np.array(y).reshape(file_number, 1)
    return X, y

# Load Data
print('DATA LOADER BEGINS')
data_dir = "/content/drive/MyDrive/DATASET"
X, y = load_data(data_dir)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print("DATA LOADED")

# Neural Network Model
model = Sequential()

# Convolutional Layer to Handle Matrix A
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape= (input_shape,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

# LSTM Layer for Time Series Data
model.add(LSTM(25, return_sequences=True))
model.add(Flatten())

# Dense Layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the Model
history = model.fit(X_train, y_train, epochs=10, batch_size=8)

# Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print("******************************************************************************************************************************")
print(f'TEST ACCURACY: {accuracy:.4f}')

# Save the Model
model.save("/content/drive/MyDrive/seizure_detection_model.h5")
print('MODEL COMPILED AND SAVED')

# Graph of Loss Against Epochs 
plt.plot(history.history['loss'], label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Function vs. Epoch')
plt.legend()
plt.show()


In [1]:
15*8*300


36000